# MNIST with Tensorflow Serving

Let's train a model first

In [1]:
# !pip install tensorflow==2.3.0
# !pip install keras==2.4.3
# !pip install pydot==1.4.2

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import (Input, Model, layers, metrics, utils, models)

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

Number of original training examples: 60000
Number of original test examples: 10000


In [20]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [22]:
def get_model(IMAGE_SHAPE):
    """
    creates and returns a simple CNN model.
    """
    # Define the tensors for the two input images
    input_twin = Input(IMAGE_SHAPE)
    x = layers.Conv2D(512, (2, 2), activation="relu")(input_twin)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, (2, 2), activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (1, 1), activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(10, name="final_dense", activation='softmax')(x)
    return Model(input_twin, outputs)

In [25]:
INPUT_SHAPE = (28, 28, 1)
model = get_model(INPUT_SHAPE)

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 512)       2560      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 512)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 256)       524544    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 128)         32896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 128)         0   

In [27]:
_learning_rate = 1e-3
model.compile(optimizer = tf.keras.optimizers.Adam(_learning_rate), 
              loss = tf.keras.losses.categorical_crossentropy, 
              metrics = ['accuracy'])

In [13]:
model.fit(x_train, y_train, validation_split=0.3, epochs = 5)

Epoch 1/5
1313/1313 [==============================] - 307s 234ms/step - loss: 0.2168 - accuracy: 0.9315 - val_loss: 0.0945 - val_accuracy: 0.9689
Epoch 2/5
1313/1313 [==============================] - 318s 243ms/step - loss: 0.0800 - accuracy: 0.9738 - val_loss: 0.0685 - val_accuracy: 0.9783
Epoch 3/5
1313/1313 [==============================] - 299s 228ms/step - loss: 0.0619 - accuracy: 0.9804 - val_loss: 0.0619 - val_accuracy: 0.9799
Epoch 4/5
1313/1313 [==============================] - 281s 214ms/step - loss: 0.0492 - accuracy: 0.9840 - val_loss: 0.0592 - val_accuracy: 0.9817
Epoch 5/5
1313/1313 [==============================] - 313s 238ms/step - loss: 0.0401 - accuracy: 0.9866 - val_loss: 0.0534 - val_accuracy: 0.9838


In [22]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 20s 62ms/step - loss: 0.0471 - accuracy: 0.9850


[0.04712369292974472, 0.9850000143051147]

In [3]:
SAVE_PATH = 'serving'
VERSION = '1'
model.save(os.path.join(SAVE_PATH, VERSION))

The model has been trained and saved successfully!